In [18]:
import pandas as pd
import numpy as np
import requests
import re
import seaborn as sns
import matplotlib.pyplot as plt

In [20]:
#upload data
data = ('/Users/rutheverett/Downloads/www-deepcrawl-com_12-10-2020_All_Pages_basic.csv')
#select colums 
columns = ['url',
           'log_requests_total',
           'log_requests_desktop',
           'log_requests_mobile',
           'indexable',
           'http_status_code',
           'primary_page',
           'duplicate_page', 
           'search_console_total_clicks',
           'level',
           'links_in_count']
#read in data colums 
df = pd.read_csv(data , usecols=columns)
df.head(10)

In [ ]:
#Segment URLs 
segment_definitions = [
    [(r'\/blog\/'), 'Blog'],
    [(r'\/technical-seo-library\/'), 'Technical SEO Library'],
    [(r'\/hangout-library\/'), 'Hangout Library'],
    [(r'\/guides\/'), 'Guides'],
    [(r'\/case-studies\/'), 'Case Studies'],
    [(r'\/why-'), 'Solutions'],
    ]

use_segment_definitions = True

def get_segment(url):
    
    if use_segment_definitions == True:
        for segment_definition in segment_definitions:
            if re.findall(segment_definition[0], url):
                return segment_definition[1]
        return 'Other'

df['segment'] = df['url'].apply(lambda x: get_segment(x))

# Replace NaN with 0
df['search_console_total_clicks'].replace(np.nan, 0, inplace=True)
df['log_requests_total'].replace(np.nan, 0, inplace=True)
df['log_requests_mobile'].replace(np.nan, 0, inplace=True)
df['log_requests_desktop'].replace(np.nan, 0, inplace=True)
df['level'].replace(np.nan, 0, inplace=True)

In [ ]:
#Create pivot table with total log file requests for each segment
total_log_requests = df.pivot_table(index='segment', values=['url', 'log_requests_total', 'log_requests_mobile', 'log_requests_desktop'], aggfunc={'url':len, 'log_requests_total':sum, 'log_requests_mobile':sum, 'log_requests_desktop':sum})
total_log_requests['percent_of_total_log_requests']= (total_log_requests['log_requests_total']/total_log_requests['log_requests_total'].sum()).apply('{:.2%}'.format)
#Reset Index
new_total_log_requests = total_log_requests.reset_index('segment')
new_total_log_requests

In [ ]:
#Create bar chart for total requests per category
sns.barplot(data=new_pivot, x='segment', y='log_requests_total')
plt.xticks(rotation=90)
plt.show()

In [ ]:
#Create bar chart for total mobile requests per category
sns.barplot(data=new_pivot, x='segment', y='log_requests_mobile')
plt.xticks(rotation=90)
plt.show()

In [ ]:
#Create bar chart for total desktop requests per category
sns.barplot(data=new_pivot, x='segment', y='log_requests_desktop')
plt.xticks(rotation=90)
plt.show()

In [ ]:
#Create pivot table with total log file requests for indexability status 
indexable_log_file = df.pivot_table(index='indexable', values=['url', 'log_requests_total', 'log_requests_mobile', 'log_requests_desktop'], aggfunc={'url':len, 'log_requests_total':sum, 'log_requests_mobile':sum, 'log_requests_desktop':sum})
indexable_log_file['percent_of_total_log_requests']= (indexable_log_file['log_requests_total']/indexable_log_file['log_requests_total'].sum()).apply('{:.2%}'.format)
indexable_log_file

In [ ]:
# Function to create a pivot table with a variable number of indexes

def pivot_table(df, indexes):
    pivot_table = df.pivot_table(index=indexes, values=['url', 'log_requests_total'], aggfunc={'url':len, 'log_requests_total':sum})

    pivot_table = pivot_table.sort_values('log_requests_total', ascending=False)
    pivot_table['percent_of_total_log_requests']= (pivot_table['log_requests_total']/pivot_table['log_requests_total'].sum()).apply('{:.2%}'.format)
    pivot_table['average_log_requests_per_url']= (pivot_table['log_requests_total']/pivot_table['url']).apply('{:.1f}'.format)
    pivot_table['log_requests_total'] = (pivot_table['log_requests_total']).apply('{:,.0f}'.format)
    pivot_table['url'] = (pivot_table['url']).apply('{:,}'.format)
    pivot_table = pivot_table.rename({'url': 'url_count'}, axis='columns')

    return pivot_table

In [ ]:
#Create pivot table with total log file requests for status code
pivot_status = pivot_table(df, ['category', 'http_status_code'])
pivot_status

In [ ]:
#Create pivot table with total log file requests for 200 status code, indexable pages - you can change these to be any status code and either indexable or non-indexable
df_200 = df[(df.http_status_code == 200)]

pivot_indexable = pivot_table(df_200, ['category', 'indexable'])
pivot_indexable

If you would like to see how log file requests have changed over 6 months: 

In [ ]:
#upload multiple dfs 
june_df = pd.read_csv(june , usecols=columns)
july_df = pd.read_csv(july , usecols=columns)
aug_df = pd.read_csv(aug , usecols=columns) 
sep_df = pd.read_csv(sep , usecols=columns)
oct_df = pd.read_csv(oct , usecols=columns)
nov_df = pd.read_csv(nov , usecols=columns)

june_df['month'] = 'May'
july_df['month'] = 'June'
aug_df['month'] = 'July'
sep_df['month'] = 'August'
oct_df['month'] = 'September'
nov_df['month'] = 'October'

#concatenate dataframes together
df_all = pd.concat(objs =[june_df, july_df, aug_df, sep_df, oct_df, nov_df], ignore_index=True)

In [ ]:
#create pivot table to store all data
def pivot_table(df, indexes):
    pivot_table = df.pivot_table(index=indexes, values=['url', 'log_requests_total'], aggfunc={'url':len, 'log_requests_total':sum})

    pivot_table = pivot_table.sort_values('log_requests_total', ascending=False)
    pivot_table['percent_of_total_log_requests']= (pivot_table['log_requests_total']/pivot_table['log_requests_total'].sum()).apply('{:.2%}'.format)
    pivot_table['average_log_requests_per_url']= (pivot_table['log_requests_total']/pivot_table['url']).apply('{:.1f}'.format)
    pivot_table['log_requests_total'] = (pivot_table['log_requests_total']).apply('{:,.0f}'.format)
    pivot_table['url'] = (pivot_table['url']).apply('{:,}'.format)
    pivot_table = pivot_table.rename({'url': 'url_count'}, axis='columns')

    return pivot_table

In [ ]:
#create pivot table to get count of log file requests for each month
pivot_table_total = df.pivot_table(index='month', values=['url', 'log_requests_total', 'log_requests_mobile', 'log_requests_desktop'], aggfunc={'url':len, 'log_requests_total':sum, 'log_requests_mobile':sum, 'log_requests_desktop':sum})
pivot_table_total 

#reset index to get months in order
new_index = ['May', 'June', 'July', 'August', 'September', 'October']
new_pivot = pivot_table_total.reindex(new_index)
new_pivot

In [ ]:
#create line graph to show log requests over time 
sns.lineplot(x = "month", y = "log_requests_total", data=new_pivot)
plt.show()